In [1]:
import copy
from dgl import batch, unbatch
import rdkit.Chem as Chem
from rdkit import RDLogger
import torch
torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
np.random.seed(0)

import os,sys,inspect
sys.path.insert(0,'/home/icarus/app/src') 

from models.jtnn_vae import DGLJTNNVAE
from models.modules import *

#from .nnutils import cuda, move_dgl_to_cuda

#from .jtnn_enc import DGLJTNNEncoder
#from .jtnn_dec import DGLJTNNDecoder
#from .mpn import DGLMPN
#from .mpn import mol2dgl_single as mol2dgl_enc
#from .jtmpn import DGLJTMPN
#from .jtmpn import mol2dgl_single as mol2dgl_dec

import os,sys,inspect
from tqdm import tqdm
sys.path.insert(0,'/home/icarus/T-CVAE-MolGen/src')

from utils.chem import set_atommap, copy_edit_mol, enum_assemble_nx, \
                            attach_mols_nx, decode_stereo

lg = RDLogger.logger()

lg.setLevel(RDLogger.CRITICAL)

RDKit WARNING: [08:27:55] Enabling RDKit 2019.09.1 jupyter extensions


In [2]:
class TCVAE(nn.Module):
    def __init__(self, vocab, hidden_size, latent_size, depth):
        super(TCVAE, self).__init__()
        self.vocab = vocab
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.depth = depth
        
        self.embedding = nn.Embedding(vocab.size(), hidden_size).cuda()
        self.mpn = MPN(hidden_size, depth)
        self.encoder = None  #
        self.decoder = None  #
        
        self.T_mean, self.T_var = nn.Linear(hidden_size, latent_size // 2), \
                                  nn.Linear(hidden_size, latent_size // 2)
        self.G_mean, self.G_var = nn.Linear(hidden_size, latent_size // 2), \
                                  nn.Linear(hidden_size, latent_size // 2)
            
        self.n_nodes_total = 0
        self.n_edges_total = 0
        self.n_passes = 0

## Posterior

As in <cite data-cite="7333468/6Y976JUQ"></cite>

$q(z|x,y) \sim N(\mu,\sigma^2I)$, where

$\quad\quad h = \text{MultiHead}(c,E_\text{out}^L(x;y),E_\text{out}^L(x;y))$

$\quad\quad \begin{bmatrix}\mu\\\log(\sigma^2)\end{bmatrix} = hW_q+b_q$

In [3]:
    def sample_posterior(self, prob_decode=False):
        return

### Prior

As in <cite data-cite="7333468/6Y976JUQ"></cite>

$p_\theta (z|x) \sim N(\mu', \sigma'^2 I)$, where:

$\quad\quad h' = \text{MultiHead}(c, E_\text{out}^L(x), E_\text{out}^L(x))$

$\quad\quad \begin{bmatrix}\mu'\\\log(\sigma'^2)\end{bmatrix} = MLP_p(h')$

In [4]:
    def sample_prior(self, prob_decode=False):
        return

In [5]:
from utils.data import JTNNDataset, JTNNCollator
#torch.multiprocessing.set_sharing_strategy('file_system')
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

class ArgsTemp():
    def __init__(self, hidden_size, depth, device):
        self.hidden_size = hidden_size
        self.batch_size = 350
        self.latent_size = 56
        self.depth = depth
        self.device = device
        self.lr = 1e-3
        self.beta = 1.0
        self.use_cuda = torch.cuda.is_available()
        
args = ArgsTemp(200,3, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
print(args.depth)

dataset = JTNNDataset(data='train', vocab='vocab', training=True, intermediates=False)
vocab = dataset.vocab
"""
dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=1,
        collate_fn=JTNNCollator(vocab, True, intermediates=False),
        drop_last=True,
        worker_init_fn=None)
        """

model = DGLJTNNVAE(vocab, args.hidden_size, args.latent_size, args.depth, args).cuda()
model.share_memory()
#if torch.cuda.device_count() > 1:
  #print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  #model = nn.DataParallel(model)
optimizer = optim.Adam(model.parameters(), lr=args.lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, 0.9)


3
Loading data...
Loading finished.
	Num samples: 220011
	Vocab file: /home/icarus/.dgl/jtnn/vocab.txt
Initializing Embedding
Initializing MPN
Initializing JTNN
Initializing Decoder
Initializing JTMPN


In [6]:
MAX_EPOCH = 50
PRINT_ITER = 20

from tqdm import tqdm
from os import access, R_OK
from os.path import isdir
import sys

save_path = '/home/icarus/app/data/05_model_output'
assert isdir(save_path) and access(save_path, R_OK), \
       "File {} doesn't exist or isn't readable".format(save_path)

def train():
    dataset.training = True
    print("Loading data...")
    dataloader = DataLoader(
            dataset,
            batch_size=args.batch_size,
            shuffle=True,
            pin_memory=True,
            num_workers=12,
            collate_fn=JTNNCollator(vocab, True),
            drop_last=True,
            worker_init_fn=None)
    dataloader._use_shared_memory = False
    last_loss = sys.maxsize
    print("Beginning Training...")
    for epoch in range(MAX_EPOCH):
        word_acc,topo_acc,assm_acc,steo_acc = 0,0,0,0
        print("Epoch %d: " % epoch)

        for it, batch in tqdm(enumerate(dataloader),total=len(dataloader)):
            model.zero_grad()
            try:
                loss, kl_div, wacc, tacc, sacc, dacc = model(batch, args.beta)
            except:
                print([t.smiles for t in batch['mol_trees']])
                raise
            loss.backward()
            optimizer.step()

            word_acc += wacc
            topo_acc += tacc
            assm_acc += sacc
            steo_acc += dacc

            cur_loss = loss.item()
            
            if (it + 1) % PRINT_ITER == 0:
                word_acc = word_acc / PRINT_ITER * 100
                topo_acc = topo_acc / PRINT_ITER * 100
                assm_acc = assm_acc / PRINT_ITER * 100
                steo_acc = steo_acc / PRINT_ITER * 100

                print("KL: %.1f, Word: %.2f, Topo: %.2f, Assm: %.2f, Steo: %.2f, Loss: %.6f, Delta: %.6f" % (
                    kl_div, word_acc, topo_acc, assm_acc, steo_acc, cur_loss, last_loss - cur_loss))
                word_acc,topo_acc,assm_acc,steo_acc = 0,0,0,0
                sys.stdout.flush()

            if (it + 1) % 1500 == 0: #Fast annealing
                scheduler.step()
                print("learning rate: %.6f" % scheduler.get_lr()[0])
                
            if (it + 1) % 100 == 0:
                torch.save(model.state_dict(),
                            save_path + "/model.iter-%d-%d" % (epoch, it + 1))
                      
            #if last_loss - cur_loss < 1e-5:
            #    break
            last_loss = cur_loss

        scheduler.step()
        print("learning rate: %.6f" % scheduler.get_lr()[0])
        torch.save(model.state_dict(), save_path + "/model.iter-" + str(epoch))

In [7]:
import warnings; warnings.simplefilter('ignore')
train() 

Loading data...
Beginning Training...
Epoch 0: 


  3%|▎         | 19/628 [03:48<1:09:03,  6.80s/it]

KL: 0.1, Word: 20.00, Topo: 73.63, Assm: 31.93, Steo: 27.29, Loss: 66.690994, Delta: -1.021431


  6%|▌         | 39/628 [07:12<3:45:28, 22.97s/it]

KL: 0.8, Word: 30.11, Topo: 84.75, Assm: 41.35, Steo: 31.33, Loss: 56.042809, Delta: 0.124046


...

  9%|▉         | 59/628 [09:21<41:11,  4.34s/it]  

KL: 1.0, Word: 60.70, Topo: 95.11, Assm: 79.98, Steo: 33.74, Loss: 29.236412, Delta: -0.020956


 11%|█         | 67/628 [10:39<51:53,  5.55s/it]  

['O=C(COC(=O)C[C@@H]1C[C@H]2CC[C@@H]1C2)Nc1cccc(Cl)c1', 'COc1cc2[nH]c(O)c(C(=O)NCCC[NH+](C)C)c(=O)c2cc1OC', 'Nc1ccnc(CN(Cc2ccncc2)C2CC2)n1', 'NC(=O)c1ccc(/C=C/C(=O)Nc2cccc([N+](=O)[O-])c2)cc1', 'COc1ccc(-c2ocnc2C(=O)Nc2ccc3c(c2)CCC3)cc1', 'Cc1cc(NC(=O)CN2CCN(Cc3cc(-c4ccc(Cl)cc4)no3)CC2)no1', 'O=C([O-])c1ccc(CN2C(=O)S[C@@H](Nc3ccc(C(=O)[O-])cc3)C2=O)cc1', 'CC[C@H]([NH2+][C@@H](CO)c1ccccc1F)c1cccs1', 'Cc1ccc2ncc(C[NH+]3CCC(Nc4ccccc4)CC3)n2c1', 'C[C@@H]1CC[C@H](C(=O)O[C@H](C(F)(F)F)C(C)(C)c2ccccc2)O1', 'CN(CC(=O)NCC[NH+](C)C)S(=O)(=O)c1cc(Cl)ccc1Cl', 'COc1cc(C(=O)N(C2CCCC2)C2CC2)cc(OC)c1OC(F)F', 'COc1ccc(CNC(=O)CSc2nnc3c(=O)n(-c4ccc(F)cc4)ccn23)cc1', 'C[C@H](C(=O)NC(=O)NC12CC3CC(CC(C3)C1)C2)N1CC[NH+](CC2CC2)CC1', 'O=C(CSc1ccc(F)cc1)N[C@H]1CCS(=O)(=O)C1', 'CCCCC1([C@H]2CC(=O)N(c3ccccc3)C2=O)C(=O)N(c2ccccc2)N(c2ccccc2)C1=O', 'CC(C)CNC(=O)C(=O)Nc1ccccc1N(C)Cc1ccccc1', 'CCOc1ccccc1NC(=O)C1CCN(S(=O)(=O)c2cc(-c3noc(C)n3)cs2)CC1', 'O=C(Cn1cnc(-c2ccccc2)cc1=O)Nc1nc2ccccc2s1', 'Cc1cccc(CNS(=O)(=O)

KeyboardInterrupt: 

## References

<div class="cite2c-biblio"></div>

In [ ]:
def test():
    dataset.training = False
    dataloader = DataLoader(
            dataset,
            batch_size=1,
            shuffle=False,
            num_workers=0,
            collate_fn=JTNNCollator(vocab, False),
            drop_last=True,
            worker_init_fn=None)#worker_init_fn)

    # Just an example of molecule decoding; in reality you may want to sample
    # tree and molecule vectors.
    for it, batch in enumerate(dataloader):
        #print(batch['mol_trees'])
        gt_smiles = batch['mol_trees'][0].smiles
        #print(gt_smiles)
        model.move_to_cuda(batch)
        _, tree_vec, mol_vec = model.encode(batch)
        tree_vec, mol_vec, _, _ = model.sample(tree_vec, mol_vec)
        smiles = model.decode(tree_vec, mol_vec)
        print(smiles)

In [ ]:
torch.cuda.empty_cache()
test()